This notebooks shows how to pretrain any language model easily


1. Pretrain Roberta Model: this notebook
2. Finetune Roberta Model: https://www.kaggle.com/maunish/clrp-pytorch-roberta-finetune<br/>
   Finetune Roberta Model [TPU]: https://www.kaggle.com/maunish/clrp-pytorch-roberta-finetune-tpu
3. Inference Notebook: https://www.kaggle.com/maunish/clrp-pytorch-roberta-inference
4. Roberta + SVM: https://www.kaggle.com/maunish/clrp-roberta-svm


In [1]:
EPOCHS = 20
EVAL_STEPS = 300
MAX_LEN = 256
BATCH_SIZE = 8

In [2]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from transformers import (AutoModel,AutoModelForMaskedLM, 
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments,
                          ElectraModel, ElectraConfig)

In [3]:
train_data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

data = pd.concat([train_data,test_data])
data['excerpt'] = data['excerpt'].apply(lambda x: x.replace('\n',''))

text  = '\n'.join(data.excerpt.tolist())

with open('text.txt','w') as f:
    f.write(text)

In [4]:
model_name = 'google/electra-large-discriminator'
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained('./clrp_electra_large');

Downloading:   0%|          | 0.00/668 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForMaskedLM: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['generator_predictions.LayerNorm.weight', 'generator_predictions.La

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

In [5]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention train text file here
    block_size=MAX_LEN)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention valid text file here
    block_size=MAX_LEN)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="./clrp_electra_large_chk", #select model path for checkpoint
    overwrite_output_dir=True,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    evaluation_strategy= 'steps',
    save_total_limit=2,
    eval_steps=EVAL_STEPS,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end =True,
    prediction_loss_only=True,
    report_to = "none")

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset)

In [6]:
trainer.train()
trainer.save_model(f'./clrp_electra_large')

Step,Training Loss,Validation Loss,Runtime,Samples Per Second
300,No log,7.022404,90.930100,31.244000
600,7.179900,6.951930,90.849000,31.272000
900,7.179900,6.927874,90.967000,31.231000
1200,6.957700,6.889078,90.956200,31.235000
1500,6.909500,6.873237,90.921800,31.247000
1800,6.909500,6.857979,90.898100,31.255000
2100,6.891400,6.859420,90.972800,31.229000
2400,6.891400,6.841524,90.962600,31.233000
2700,6.860900,6.842601,90.954400,31.235000
3000,6.869300,6.837497,90.904900,31.252000
